<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/PyTTI_Tools_FiLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FiLM：ラージモーション用フレーム補間機能**
このノートブックは、GoogleのML研究者によって公開された学習済みモデルを便利に使うためのラッパーです。彼らの研究についてはこちらをご覧ください。

* https://github.com/google-research/frame-interpolation

---

David Marx(@DigThatData) が作成したノートブックで、MITライセンスの下で公開されています。

このノートブックに関して何か問題が発生した場合、または提案がある場合は、ここで報告してください。
* https://github.com/pytti-tools/pytti-notebook/issues/new 


In [1]:
#1.ワークスペースの設定
#このセルを実行すると、セットアップが行われます。
#Googleドライブのマウントはオプションです。 ドライブをマウントすると、コードやモデルがそこにダウンロードされます。 これにより、初回実行時のセットアップ時間が短縮されます。
from pathlib import Path
import os

drive_mounted = False
gdrive_fpath = '.'
local_path = '/content/'

##############################

#@markdown # 1.作業空間の設定

#@markdown このセルを実行すると、設定が行われます。

#@markdown - Google記憶装置の接続は選択制です。 
#@markdown - 記憶装置に接続すると、コードやモデルがそこにダウンロードされます。
#@markdown - これにより、初回実行時の設定時間が短縮されます。
#@markdown ---

###################


# 選択制でGDriveを搭載可能
#@markdown - **グーグル記憶装置をアクセスしているか**
mount_gdrive = True # @param{type:"boolean"}
if mount_gdrive and not drive_mounted:
    from google.colab import drive
#@markdown - **グーグル記憶装置の場所**
    gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
#@markdown - **グーグル記憶装置のファイル場所**
    gdrive_subdirectory = 'MyDrive/interpolation' #@param{type:"string"}
    gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
    try:
        drive.mount(gdrive_mountpoint, force_remount = True)
        !mkdir -p {gdrive_fpath}
        %cd {gdrive_fpath}
        local_path = gdrive_fpath
        drive_mounted = True
    except OSError:
        print(
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            "OSErrorが発生しても、記憶装置がすでに接続されていれば、無視してください。"
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            )
        raise

#####################

# 残りの設定を行う

if not Path('./frame-interpolation').exists():
    !git clone https://github.com/pytti-tools/frame-interpolation

try:
    import frame_interpolation
except ModuleNotFoundError:
    %pip install -r ./frame-interpolation/requirements_colab.txt
    %pip install ./frame-interpolation

#url = "https://drive.google.com/drive/folders/1GhVNBPq20X7eaMsesydQ774CgGcDGkc6?usp=sharing"
share_id = "1GhVNBPq20X7eaMsesydQ774CgGcDGkc6"

if not (Path(local_path) / 'saved_model').exists():
    !pip install --upgrade gdown
    !gdown --folder {share_id}

# 初期のフレームを作成する
!mkdir -p frames

Mounted at /content/drive/
/content/drive/MyDrive/interpolation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.7 MB 37.2 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |████████████████████████████████| 151 kB 74.8 MB/s 
     |████████████████████████████████| 2.7 MB 63.9 MB/s 
     |████████████████████████████████| 15.4 MB 68.4 MB/s 
     |████████████████████████████████| 23.6 MB 1.2 MB/s 
     |████████████████████████████████| 526 kB 73.2 MB/s 
     |████████████████████████████████| 829 kB 71.8 MB/s 
     |████████████████████████████████| 62 kB 49 kB/s 
     |████████████████████████████████| 278 kB 78.0 MB/s 
     |████████████████████████████████| 1.6 MB 65.3 MB/s 
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43512 sha256=c30aa5d55695b6f03c2c1ee83c29027c5e14778fd0aab3d1e84bbe3c58407574
  Stored in directory: /root/.cache/pi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./frame-interpolation
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyttitools-film: filename=pyttitools_film-0.0.1-py3-none-any.whl size=54530 sha256=5ac16934c8f94b636da8b77031854105e3af2c78e0f3e58f2ac44f1951bfb38c
  Stored in directory: /root/.cache/pip/wheels/d7/8c/02/43f17147bb062ce7ca3cfd393e1064ad2d774e98fd6922f8ab
Successfully built pyttitoo

In [2]:
#@markdown # 2.セットアップ 補間

#@markdown frames_dirパラメータで、ビデオフレームを格納するディレクトリを指定します。

frames_dir = "frames" #@param{'type':'string'}

#@markdown 1回の補間処理で、frames_dirにある連続したフレームのペアの間にフレームが追加されます。

#@markdown n  frames_dir のフレームから始めて 、recursive_interpolation_passes を  k  とすると、補間後の総フレーム数は、2k(n-1)-1になります。
#@markdown $$2^k (n-1) -1$$
#@markdown - **recursive_interpolation_passes(再帰補間パス)**
recursive_interpolation_passes = 1 #@param{'type':'integer'}

#@markdown ビデオ出力を行う場合は、このボックスにチェックを入れます。
#@markdown ビデオ出力が発生しない場合は、FPSオプションを無視することができます。
output_video = True #@param{'type':'boolean'}
output_video_fps = 30 #@param{'type':'number'}

In [3]:
#@markdown # 3. 補間せよ
#@markdown 結果は、「interpolated_frames(補間フレーム)」という名前の frames_dir のサブディレクトリに保存されます

from loguru import logger

logger.info("Beginning interpolation...")

if output_video:
  !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \
      --fps {output_video_fps} \
      --output_video
      

else:
    !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \

logger.info("Interpolation comlpete.")

2022-12-22 01:00:11.464 | INFO     | __main__:<module>:6 - Beginning interpolation...


2022-12-22 01:00:16.402 | DEBUG    | __main__:_run_pipeline:171 - ['frames']
W1222 01:00:16.465470 139812601735040 environments.py:371] Make sure that locally built Python SDK docker image has Python 3.8 interpreter.
I1222 01:00:16.465632 139812601735040 environments.py:380] Default Python SDK image for environment is apache/beam_python3.8_sdk:2.34.0
I1222 01:00:16.491399 139812601735040 translations.py:678] ==================== <function annotate_downstream_side_inputs at 0x7f2820c0c670> ====================
I1222 01:00:16.491640 139812601735040 translations.py:678] ==================== <function fix_side_input_pcoll_coders at 0x7f2820c0c790> ====================
I1222 01:00:16.491766 139812601735040 translations.py:678] ==================== <function pack_combiners at 0x7f2820c0cca0> ====================
I1222 01:00:16.492036 139812601735040 translations.py:678] ==================== <function lift_combiners at 0x7f2820c0cd30> ====================
I1222 01:00:16.492156 139812601735040

2022-12-22 01:00:29.174 | INFO     | __main__:<module>:14 - Interpolation comlpete.


MITライセンス
 
Copyright (c) 2022 David Marx
 
 本ソフトウェアおよび関連文書ファイルのコピー (以下「ソフトウェア」)を取得するすべての人に、本ソフトウェアを無制限に 取引する許可をここに与えます。

本ソフトウェアおよび関連文書ファイル（以下「本ソフトウェア」）のコピー
、本ソフトウェアの
、無制限に取引することができます。
のコピーを使用、コピー、修正、結合、出版、配布、 サブライセンス、および/または販売する権利（
 ）を無制限に含みます。
 
 また、本ソフトウェアが
に提供された者に対して、以下の条件に従って、本ソフトウェアを提供することを許可します。

上記の著作権表示およびこの許諾表示は、本ソフトウェアのすべての
コピーまたは相当部分に含まれるものとします。


本ソフトウェアは「現状のまま」提供され、 明示または黙示を問わず、以下のようないかなる種類の保証も伴いません。

ただし、商品性、
特定目的への適合性および非侵害の保証に限定されるものではありません。

いかなる場合においても、
の著者または著作権者は、責任を負いません。

請求、損害賠償、その他
の責任について、それが訴訟で あるか否かを問わず。契約、不法行為 、その他に起因する、

に起因する、または関連する。本ソフトウェア、またはソフトウェアの使用またはその他の取引に関するもの。